In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter


# 1. Load Document

In [4]:
loader = PyPDFLoader("docs/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf")
pages = loader.load()

In [5]:
len(pages)

107

In [6]:
len(pages[0].page_content)

2912

In [10]:
print(pages[0].page_content[0:2000])

 
1 
  
TEXTO APROBADO POR EL CONSEJO CONSTITUCIONAL  
 
 
CAPÍTULO I  
 
FUNDAMENTOS DEL ORDEN CONSTITUCIONAL  
 
Artículo 1  
 
1. La dignidad humana es inviolable y la base del derecho y la justicia. Las personas nacen 
libres e iguales en dignidad y derechos. Su respeto y garantía es el primer deber de la 
comunidad política y de su forma jurídica de organización.  
 
2. La familia es el núcleo fundamental de la sociedad. Es deber del Estado y la sociedad dar 
protección a las familias y propender a su for talecimiento.  
 
3. Las agrupaciones que libremente surjan entre las personas gozarán de la adecuada 
autonomía para cumplir sus fines específicos que no sean contrarios a la Constitución. El 
Estado respetará los efectos de este reconocimiento . 
 
Artículo 2  
 
1. El Estado de Chile es social y democrático de derecho, que reconoce  derechos y libertades 
fundamentales, deberes constitucionales, y promueve el desarrollo progresivo de los 
derechos sociales, con sujeción al pri

In [8]:
pages[0].metadata

{'source': 'docs/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf',
 'page': 0}

# 2. Split Document (1500, 300)

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "\. ", " ", ""],
    chunk_size=2000,
    chunk_overlap=1000,
    length_function=len
)

In [12]:
docs = text_splitter.split_documents(pages)

In [13]:
len(docs)

288

In [14]:
print(docs[0].page_content), print(docs[1].page_content)

1 
  
TEXTO APROBADO POR EL CONSEJO CONSTITUCIONAL  
 
 
CAPÍTULO I  
 
FUNDAMENTOS DEL ORDEN CONSTITUCIONAL  
 
Artículo 1  
 
1. La dignidad humana es inviolable y la base del derecho y la justicia. Las personas nacen 
libres e iguales en dignidad y derechos. Su respeto y garantía es el primer deber de la 
comunidad política y de su forma jurídica de organización.  
 
2. La familia es el núcleo fundamental de la sociedad. Es deber del Estado y la sociedad dar 
protección a las familias y propender a su for talecimiento.  
 
3. Las agrupaciones que libremente surjan entre las personas gozarán de la adecuada 
autonomía para cumplir sus fines específicos que no sean contrarios a la Constitución. El 
Estado respetará los efectos de este reconocimiento . 
 
Artículo 2  
 
1. El Estado de Chile es social y democrático de derecho, que reconoce  derechos y libertades 
fundamentales, deberes constitucionales, y promueve el desarrollo progresivo de los 
derechos sociales, con sujeción al princ

(None, None)

# 3. Create Embeddings

In [18]:
from langchain.embeddings import BedrockEmbeddings 
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

In [19]:
bedrock_embeddings.embed_query(text="Hola Mundo")[:3]

[0.51953125, 0.44921875, 0.32421875]

In [20]:
from langchain.vectorstores import Chroma

In [21]:
persist_directory = 'streamlit/docs/chroma/'
!rm -rf .streamlit/docs/chroma  # remove old database files if any

In [22]:
%%time
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=bedrock_embeddings,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

596
CPU times: user 2.22 s, sys: 327 ms, total: 2.55 s
Wall time: 1min 29s


In [23]:
%%time
question = "¿Cómo se compone el Congreso Nacional según la propuesta de nueva constitución chilena?"
responses = vectordb.similarity_search(question,k=8)

CPU times: user 15.7 ms, sys: 3.84 ms, total: 19.5 ms
Wall time: 306 ms


In [25]:
vectordb.persist()

In [26]:
print(vectordb._collection.count())

596


In [28]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=8)

In [29]:
for doc_mmr in docs_mmr:
    print (doc_mmr.page_content)

95 
  
el Presidente de la República convoque a referendo, se promulgará el proyecto que hubiere 
aprobado el Congreso Nacional.  
 
2. El decreto de convocatoria contendrá, según corresponda, el proyecto aprobado por ambas 
Cámaras y vetado totalmente por e l Presidente de la República, o las cuestiones del proyecto 
en las cuales el Congreso haya insistido, según lo dispuesto en los incisos 4 y 5 del artículo 
anterior. En este último caso, cada una de las cuestiones en desacuerdo deberá ser votada 
separadamen te en el referendo.  
 
3. El Tribunal Calificador de Elecciones comunicará al Presidente de la República el 
resultado del referendo y especificará el texto del proyecto aprobado por la ciudadanía, el 
que deberá ser promulgado como reforma constitucional dent ro de los cinco días siguientes 
a dicha comunicación.  
 
4. Una vez promulgado el proyecto y desde la fecha de su vigencia, sus disposiciones 
formarán parte de la Constitución y se tendrán por incorporadas a esta.  
